### This is the Notebook for Owen Monroe's Homework Assigment 2 for IS-567 Text Mining
##### September 23, 2023

1. Loading the Dataset

In [79]:
import pandas as pd
import numpy as np
import nltk
from nltk import pos_tag

In [18]:
train_df = pd.read_csv('chatgpt_train.csv')
test_df = pd.read_csv('chatgpt_test.csv')
train_df['review'][24]

nan

In [37]:
# Creates lists and reporting length

tr_rev_list = train_df['review'].to_list()
te_rev_list = test_df['review'].to_list()
tr_rev_list
te_rev_list
print(len(tr_rev_list))
print(len(te_rev_list))


1834
458


In [75]:
# Removing Empties and reporting length
full_train_df = train_df.dropna()
full_test_df = test_df.dropna()

print(len(full_train_df['review'].to_list()))
print(len(full_test_df['review'].to_list()))

1829
458


2. POS Tagging

In [84]:
# Copying training Data and POS tagging
train_task2_df = full_train_df

# Tokenizing the Reviews
train_task2_df['NLTK Tokenized Reviews'] = train_task2_df['review'].apply(nltk.word_tokenize)

train_task2_df

/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_45660/1858178041.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_task2_df['NLTK Tokenized Reviews'] = train_task2_df['review'].apply(nltk.word_tokenize)


,date,title,review,rating,NLTK Tokenized Reviews
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4,"[Up, to, this, point, I, ?, €, ?, ve, mostly, ..."
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4,"[I, ?, €, ?, ve, been, a, user, since, it, ?, ..."
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4,"[This, app, would, almost, be, perfect, if, it..."
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4,"[I, recently, downloaded, the, app, and, overa..."
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4,"[I, appreciate, the, devs, implementing, Siri,..."
...,...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1,"[Please, make, a, iPad, version, of, this]"
1830,5/19/2023 1:23,Goated app,Best app,5,"[Best, app]"
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1,"[Why, it, ?, €, ?, s, not, available, in, Ethi..."
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1,"[It, agrees, with, letting, children, be, forc..."


In [85]:
# Library Download and POS Tagging

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

train_task2_df['POS Tagged Tokens'] = train_task2_df['NLTK Tokenized Reviews'].apply(pos_tag)
train_task2_df

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/owenmonroe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/owenmonroe/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/var/folders/8c/38d5fhqx2_jf385d_b3cq3j00000gn/T/ipykernel_45660/1173585453.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_task2_df['POS Tagged Tokens'] = train_task2_df['NLTK Tokenized Reviews'].apply(pos_tag)


,date,title,review,rating,NLTK Tokenized Reviews,POS Tagged Tokens
0,5/21/2023 16:42,Much more accessible for blind users than the ...,Up to this point I?€?ve mostly been using Chat...,4,"[Up, to, this, point, I, ?, €, ?, ve, mostly, ...","[(Up, RB), (to, TO), (this, DT), (point, NN), ..."
1,7/11/2023 12:24,"Much anticipated, wasn?€?t let down.",I?€?ve been a user since it?€?s initial roll o...,4,"[I, ?, €, ?, ve, been, a, user, since, it, ?, ...","[(I, PRP), (?, .), (€, VB), (?, .), (ve, FW), ..."
2,5/19/2023 10:16,"Almost 5 stars, but?€? no search function",This app would almost be perfect if it wasn?€?...,4,"[This, app, would, almost, be, perfect, if, it...","[(This, DT), (app, NN), (would, MD), (almost, ..."
3,5/27/2023 21:57,"4.5 stars, here?€?s why","I recently downloaded the app and overall, it'...",4,"[I, recently, downloaded, the, app, and, overa...","[(I, PRP), (recently, RB), (downloaded, VBD), ..."
4,6/9/2023 7:49,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4,"[I, appreciate, the, devs, implementing, Siri,...","[(I, PRP), (appreciate, VBP), (the, DT), (devs..."
...,...,...,...,...,...,...
1829,6/4/2023 10:50,pad os???,Please make a iPad version of this,1,"[Please, make, a, iPad, version, of, this]","[(Please, NNP), (make, VB), (a, DT), (iPad, JJ..."
1830,5/19/2023 1:23,Goated app,Best app,5,"[Best, app]","[(Best, JJS), (app, NN)]"
1831,6/21/2023 20:02,Co,Why it?€?s not available in Ethiopia,1,"[Why, it, ?, €, ?, s, not, available, in, Ethi...","[(Why, WRB), (it, PRP), (?, .), (€, VB), (?, ...."
1832,6/7/2023 10:25,Crazy world views,It agrees with letting children be forced into...,1,"[It, agrees, with, letting, children, be, forc...","[(It, PRP), (agrees, VBZ), (with, IN), (lettin..."
